In [2]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup

import pandas as pd
import requests

# Import the API key
from config import geoapify_key
from config import census_key


# Launch the browser
browser = Browser('chrome')

In [5]:
#pull state abbrevations from the state_list ditionary file in preparation 
#to create a function to pull revenue for all years for each state in the list
state_abb_df = pd.read_csv('../Resources/state_list_REFERENCE.csv')
state_abb_df.head()

,state_abbr,state
0,AK,Alaska
1,AL,Alabama
2,AR,Arkansas
3,AZ,Arizona
4,CA,California


In [6]:
url = "https://www.legalsportsreport.com/sports-betting/revenue/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Create an empty list to store all market data
all_market_data_list = []

# Iterate through each state abbreviation in the DataFrame
for state_abbr in state_abb_df['state_abbr']:
    # Construct the class name for the current state's table
    table_class = f'tablepress-id-Rev{state_abbr}'
    
    # Find the table element for the current state
    rev_table = soup.find('table', class_=table_class)
    
    # Check if the table is found
    if rev_table:
        # Find all rows in the table body
        rows = rev_table.select('tbody tr')

        # Loop through the rows
        for row in rows:
            # Find all cells in the row
            cells = row.select('td')
            
            # Check if there are cells in the row
            if cells:
                # Extract data from the cells
                date = cells[0].text.strip()
                handle = cells[1].text.strip()
                revenue = cells[2].text.strip()
                hold = cells[3].text.strip()
                taxes = cells[4].text.strip()
                
                # Create a dictionary for the current market including the state abbreviation
                market_data = {
                    'state_abbr': state_abbr,
                    'date': date,
                    'handle': handle,
                    'revenue': revenue,
                    'hold': hold,
                    'taxes': taxes
                }
                
                # Append the market data dictionary to the list
                all_market_data_list.append(market_data)
    else:
        print(f"Table for state {state_abbr} not found.")

# Close the browser
browser.quit()

# Create a DataFrame from the list of dictionaries
all_market_data_df = pd.DataFrame(all_market_data_list)

# Print the extracted data
print(all_market_data_df)

Table for state AK not found.
Table for state AL not found.
Table for state CA not found.
Table for state FL not found.
Table for state GA not found.
Table for state HI not found.
Table for state ID not found.
Table for state KY not found.
Table for state ME not found.
Table for state MN not found.
Table for state MO not found.
Table for state NC not found.
Table for state ND not found.
Table for state NE not found.
Table for state NM not found.
Table for state OK not found.
Table for state PR not found.
Table for state SC not found.
Table for state TX not found.
Table for state UT not found.
Table for state VT not found.
Table for state WA not found.
Table for state WI not found.
     state_abbr            date       handle     revenue   hold     taxes
0            AR       July 2019     $416,160     $76,885  18.5%    $9,995
1            AR     August 2019     $778,249     $58,858   7.6%    $7,652
2            AR  September 2019   $2,252,196    $183,044   8.1%   $23,796
3            A

In [8]:
all_market_data_df.head(5)

,state_abbr,date,handle,revenue,hold,taxes
0,AR,July 2019,"$416,160","$76,885",18.5%,"$9,995"
1,AR,August 2019,"$778,249","$58,858",7.6%,"$7,652"
2,AR,September 2019,"$2,252,196","$183,044",8.1%,"$23,796"
3,AR,October 2019,"$2,216,369","$380,006",17.1%,"$49,401"
4,AR,November 2019,"$2,978,235","$181,497",6.1%,"$23,595"


Dataset format cleaning: 

In [9]:
# Replace '$', ',', and parentheses, and convert to float
def convert_value(value_str):
    value_str = value_str.replace(',', '').replace('$', '').replace('(', '-').replace(')', '')
    try:
        return float(value_str)
    except ValueError:
        return float(0)  # Return 0.0 if conversion is not possible

In [10]:
# Apply the convert_value function to all columns
all_columns = ['handle', 'revenue', 'taxes']
for column in all_columns:
    all_market_data_df[column] = all_market_data_df[column].apply(convert_value)

# Set the float format
pd.options.display.float_format = '{:.2f}'.format

In [11]:
def convert_percentage(percentage_value):
    if isinstance(percentage_value, str) and percentage_value != '':
        if percentage_value == '—':
            return 0.0
        else:
            return float(percentage_value.replace('%', '')) / 100
    else:
        return 0.0  # Handle empty strings and non-string values by returning 0.0

all_market_data_df['hold'] = all_market_data_df['hold'].apply(convert_percentage)


In [12]:
all_market_data_df.head(5)

,state_abbr,date,handle,revenue,hold,taxes
0,AR,July 2019,416160.00,76885.00,0.18,9995.00
1,AR,August 2019,778249.00,58858.00,0.08,7652.00
2,AR,September 2019,2252196.00,183044.00,0.08,23796.00
3,AR,October 2019,2216369.00,380006.00,0.17,49401.00
4,AR,November 2019,2978235.00,181497.00,0.06,23595.00


In [13]:
#load to csv:

all_market_data_df.to_csv('../Resources/all_state_market_data_FINALSET.csv', index=False)

In [ ]:
browser.quit()